In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

full_bklyn_df = pd.read_csv('brooklyn_2022_sales.csv')
full_bklyn_df.shape

(15555, 13)

In [2]:
#create full DF and combined columns for first and second mtgs into one colums, 
#added a column for count to track how many get combined later into groupbys and

full_bklyn_df = full_bklyn_df.fillna(0)
full_bklyn_df['Amount Mortgaged'] = full_bklyn_df['1st Amount'] + full_bklyn_df['2nd Amount']
full_bklyn_df['Zip Count'] = full_bklyn_df['ZIP'].map(full_bklyn_df['ZIP'].value_counts())
full_bklyn_df.tail()

,Type,Address,ZIP,Purchase Amt,Purchase Down %,1st Amount,2nd Amount,CLTV %,Purchase Date,Purchase Type,City,Owner Occ?,Listed for Sale?,Amount Mortgaged,Zip Count
15550,CND,127 PUTNAM AVE 1B,11238.0,1190000.0,25.0,892500.0,0.0,72.0,6/1/2023,Market,BROOKLYN,0.0,0.0,892500.0,709
15551,CND,108 JACKSON ST 1B,11211.0,1430000.0,20.0,1144000.0,0.0,81.0,10/26/2023,Market,BROOKLYN,0.0,0.0,1144000.0,551
15552,CND,2222 OCEAN AVE 2F,11229.0,380000.0,100.0,0.0,0.0,0.0,11/14/2023,Market,BROOKLYN,0.0,0.0,0.0,952
15553,CND,2218 OCEAN AVE 5E,11229.0,315000.0,100.0,0.0,0.0,0.0,4/25/2023,Market,BROOKLYN,0.0,0.0,0.0,952
15554,The information contained in this report is su...,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,1


In [3]:
#filtered DF to only relevant columns
bklyn_df = full_bklyn_df.loc[:, ['ZIP', 'Zip Count', 'Type', 'Owner Occ?', 'Purchase Down %', 
                                 'Purchase Amt', 'Amount Mortgaged']]
bklyn_df.head()

,ZIP,Zip Count,Type,Owner Occ?,Purchase Down %,Purchase Amt,Amount Mortgaged
0,11201.0,1187,CND,0.0,33.0,4600000.0,3062500.0
1,11201.0,1187,CND,0.0,30.0,4550000.0,3185000.0
2,11201.0,1187,CND,0.0,69.0,2620000.0,825084.0
3,11201.0,1187,CND,1.0,100.0,2700000.0,0.0
4,11201.0,1187,CND,0.0,100.0,4740000.0,0.0


In [8]:
#calculate the mortage tax as 1.8% for mortgages less than $500,000 and 1.95% for mortgages of $500,000 or more
#calculate the mansion tax as 1% for sales over $1,000,000
# calculate the hypotehtical sales tax if the tax rate as 2/3 or the mortgage tax (1.19%) for purchases less than
# $1,000,000 and add an additional 0.5% (1.69%)for purchases over $1,000,000

bklyn_df['Mtg Tx Pd'] = np.where(bklyn_df['Amount Mortgaged'] >= 500000, 
                                 bklyn_df['Amount Mortgaged'] * 0.0195, bklyn_df['Amount Mortgaged'] * 0.018)
bklyn_df['Mansion Tax Pd'] = np.where(bklyn_df['Purchase Amt'] >= 1000000, bklyn_df['Purchase Amt'] * 0.01, 0)
bklyn_df['New Sales Tax Amount'] = np.where(bklyn_df['Purchase Amt'] >= 1000000, bklyn_df['Purchase Amt'] * 0.0169,
                                            bklyn_df['Purchase Amt'] * 0.0169)

bklyn_df.head()

,ZIP,Zip Count,Type,Owner Occ?,Purchase Down %,Purchase Amt,Amount Mortgaged,Mtg Tx Pd,Mansion Tax Pd,New Sales Tax Amount
0,11201.0,1187,CND,0.0,33.0,4600000.0,3062500.0,59718.750,46000.0,77740.0
1,11201.0,1187,CND,0.0,30.0,4550000.0,3185000.0,62107.500,45500.0,76895.0
2,11201.0,1187,CND,0.0,69.0,2620000.0,825084.0,16089.138,26200.0,44278.0
3,11201.0,1187,CND,1.0,100.0,2700000.0,0.0,0.000,27000.0,45630.0
4,11201.0,1187,CND,0.0,100.0,4740000.0,0.0,0.000,47400.0,80106.0


In [7]:
#show the total mtg tax and mansion taxes collected and compare to what our hypothetical tax would have raised

total_mtg_tax_pd = bklyn_df['Mtg Tx Pd'].sum()
total_mansion_tax_pd = bklyn_df['Mansion Tax Pd'].sum()
total_est_tax = bklyn_df['New Sales Tax Amount'].sum()
print(f"The total mtg tax paid in bklyn in 2022 was ${total_mtg_tax_pd:,.0f} and the total mansion tax collected was ${total_mansion_tax_pd:,.0f}.")
print(f'The combined mtg and mansion tax was {total_mtg_tax_pd + total_mansion_tax_pd:,.0f}, our hypothetical sales tax would have raised ${total_est_tax:,.0f}')

The total mtg tax paid in bklyn in 2022 was $177,709,482 and the total mansion tax collected was $108,936,820.
The combined mtg and mansion tax was 286,646,302, our hypothetical sales tax would have raised $299,713,802


In [ ]:
bklyn_by_zip_and_occ = bklyn_df.groupby(['ZIP', 'Owner Occ?']).agg({'Zip Count': 'count', 'Purchase Amt': 'sum', 
                                                            'Amount Mortgaged': 'sum', 
                                                            'Est Mtg Tx Pd': 'sum', 'Est Sales Tax Proposed': 'sum'})

bklyn_by_zip_and_occ.head()

In [ ]:
bklyn_by_zip = bklyn_df.groupby(['ZIP']).agg({'Zip Count': 'count', 'Purchase Amt': 'sum', 'Purchase Down %': 'mean',
                                                            'Amount Mortgaged': 'sum', 
                                                            'Est Mtg Tx Pd': 'sum', 'Est Sales Tax Proposed': 'sum'})
bklyn_by_zip.shape

In [ ]:
bklyn_by_zip['Average Purchase $'] = bklyn_by_zip['Purchase Amt']/bklyn_by_zip['Zip Count']
bklyn_by_zip['Average Mtg'] = bklyn_by_zip['Amount Mortgaged']/bklyn_by_zip['Zip Count']
bklyn_by_zip['Ave Hypothetical Tax'] = bklyn_by_zip['Est Sales Tax Proposed']/bklyn_by_zip['Zip Count']
pd.options.display.float_format = '{:.1f}'.format

bklyn_by_zip.head(20)